In [1]:
import sys
sys.executable

'C:\\Users\\trang\\anaconda3\\envs\\min_ds-env\\python.exe'

In [2]:
import requests
from requests_html import HTMLSession
import time
import json
import re
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime, timedelta
import urllib

In [3]:
#cài đặt hiển thị tất cả các cột
pd.set_option('display.max_columns', None)
#set các trường dữ liệu cài đặt cho dataframe lưu dữ liệu users, playlists, tracks
u_unorder=['username','country_name','avatar_url','city','comments_count','created_at','description','followers_count','followings_count','first_name','full_name','groups_count','id','last_modified','last_name','likes_count','playlist_likes_count','permalink_url','playlist_count','track_count']
u_order=['id','username','first_name','last_name','full_name','description','avatar_url','country_name','city','created_at','last_modified','followers_count','followings_count','groups_count','comments_count','likes_count','playlist_likes_count','playlist_count','track_count','permalink_url']
p_unorder=['author','image_playlist','numTracks','created_at','description','duration','genre','id','label_name','last_modified','likes_count','permalink_url','public','release_date','reposts_count','sharing','tag_list','title','trackids','first_track']
p_order=['id','title','image_playlist','description','duration','genre','label_name','tag_list','author','created_at','release_date','last_modified','likes_count','reposts_count','public','sharing','numTracks','trackids','first_track','permalink_url']
t_unorder=['comment_count','created_at','description','download_count','duration','genre', 'id', 'label_name', 'last_modified', 'likes_count','permalink_url', 'playback_count','release_date', 'reposts_count', 'tag_list','title', 'track_format','author']
t_order=['id', 'title','description','track_format','duration','genre', 'label_name', 'tag_list','author','created_at', 'release_date','last_modified', 'comment_count','download_count', 'likes_count', 'playback_count', 'reposts_count', 'permalink_url']

In [4]:
#Hàm lấy thông tin user truyền vào địa chỉ url và DataFrame users, trả về users
def collect_user(url,users):
    session = HTMLSession()
    r = session.get(url)
    if(r):
        value=[]
        soup=BeautifulSoup(r.content,features='html.parser')
        for s in (soup.find_all("script")):
            #Xét script có dữ liệu cần tìm 
            if("hydratable" in str(s)):
                #Lấy các dữ liệu trong meta
                st=str(s).find('"username"')
                en=str(s).find('",',st)
                value.append(str(s)[st+len('"username":"'):en])
                value.append(soup.find("meta", property="og:country-name").get('content'))
                #Cắt chuỗi trong script
                i=str(s).find('{"hydratable":"user","')
                j=str(s).find('"uri"')
                s=str(s)[i+len('{"hydratable":"user","data":{'):j-1]
                #Cắt chuỗi thành một mảng gồm nhiều trường dữ liệu
                r1 = re.split(r',"',s)
                for i in r1:
                    t=re.split(r'":',i)
                    #Với mỗi phần tử là trường dữ liệu tồn tại trong dataframe, thêm vào mảng tạm value theo thứ tự
                    if(t[0].strip('"') in list(users)):
                        #Nếu t được cắt thành 2 phần tử thì thêm t[1] là giá trị vào value
                        #Nếu t được cắt thành 1 trường dữ liệu t[0] là tên của trường dữ liệu, thêm null vào value
                        #Ngược lại, nếu t rỗng thì bỏ qua
                        if(len(t)==2):
                            value.append(t[1].strip('"'))
                        elif(t[0]):
                            value.append('null')
                #Xét điều kiện kích thước của value bằng số lượng các cột của users thì thêm dữ liệu vào users
                if(len(value)==len(list(users))):
                    zipped = zip(list(users), value)
                    u = dict(zipped)
                    users=users.append(u,ignore_index=True)
    return users

In [5]:
#Kiểm tra hàm collect_user
u=pd.DataFrame(columns=u_unorder)
u=collect_user(f'https://soundcloud.com/1041',u) 
u=u[u_order]
display(u)

,id,username,first_name,last_name,full_name,description,avatar_url,country_name,city,created_at,last_modified,followers_count,followings_count,groups_count,comments_count,likes_count,playlist_likes_count,playlist_count,track_count,permalink_url
0,129504241,POU3Zz ON DA TRACK,SIUA,TUPOU,SIUA TUPOU,ALL FOR THE LOVE OF SOUNDS,https://i1.sndcdn.com/avatars-000249153357-0v7...,New Zealand,AUCKLAND,2014-12-25T10:58:11Z,2016-09-02T00:37:53Z,13,20,0,0,54,1,1,0,https://soundcloud.com/1041


In [6]:
#Hàm lấy thông tin playlist truyền vào địa chỉ url và DataFrame playlists
def collect_playlist(url,playlists):
    session = HTMLSession()
    r = session.get(url) 
    if(r):
        value=[]
        soup = BeautifulSoup(r.content,features="html.parser")
        #print(soup)
        for s in (soup.find_all("script")):
            #Xét script có dữ liệu cần tìm 
            if("hydratable" in str(s)):
                #Lấy các dữ liệu trong meta
                value.append(soup.find("meta",itemprop="name").get('content'))
                value.append(soup.find("meta", property="og:image").get('content'))
                value.append(soup.find("meta", itemprop="numTracks").get('content'))
                #Cắt chuỗi trong script
                i=str(s).find('{"hydratable":"playlist"')
                s=str(s)[i+len('{"hydratable":"playlist","data":{'):]
                j=s.find('"uri"')
                s1=s[:j-1]
                s2=s[str(s).find('"tracks":[{')+len('"tracks":[{'):]
                #Cắt chuỗi thành một mảng gồm nhiều trường dữ liệu
                r1 = re.split(r',"',s1)
                for i in r1:
                    t=re.split(r'":',i)
                    #Với mỗi phần tử là trường dữ liệu tồn tại trong dataframe, thêm vào mảng tạm value theo thứ tự
                    if(t[0].strip('"') in list(playlists)):
                        #Nếu t được cắt thành 2 phần tử thì thêm t[1] là giá trị vào value
                        if(len(t)==2): 
                            value.append(t[1].strip('"'))
                trackids="'"
                for k in range(min(5,int(value[2]))):
                    s2=s2[str(s2).find('"id":')+len('"id":'):]
                    end=str(s2).find(',"')
                    trackids = trackids + s2[:end]
                    if(k!=int(value[2])-1): 
                        trackids=trackids+","
                for k in range(0,int(value[2])-5):
                    s2=s2[str(s2).find('{"id":')+len('{"id":'):]
                    end=str(s2).find(',"')
                    trackids = trackids + s2[:end]
                    if(k!=int(value[2])-5-1): 
                        trackids=trackids+','
                trackids=trackids+"'"
                value.append(str(trackids))
                #Lấy thêm link track đầu tiên trong playlist
                value.append(f'https://soundcloud.com' + soup.find('h2',itemprop ='name').find('a',itemprop="url").get('href'))
                #Xét điều kiện kích thước của value bằng số lượng các cột của playlist thì thêm dữ liệu vào playlists
                if(len(value)==len(list(playlists))):
                    zipped = zip(list(playlists), value)
                    p = dict(zipped)
                    playlists=playlists.append(p,ignore_index=True)
    return playlists


In [7]:
#Kiểm tra hàm collect_playlist
url = f'https://soundcloud.com/1077/sets/godtierplaylist'
p = pd.DataFrame(columns=p_unorder)
p = collect_playlist(url,p)
p=p[p_order]
display(p)


,id,title,image_playlist,description,duration,genre,label_name,tag_list,author,created_at,release_date,last_modified,likes_count,reposts_count,public,sharing,numTracks,trackids,first_track,permalink_url
0,635697579,GYM,https://i1.sndcdn.com/artworks-Pif1kk65L0B9Vdk...,,4174356,,,,idk,2018-11-02T13:45:18Z,2018-11-03T00:00:00Z,2021-11-03T00:31:26Z,1,0,true,public,23,"'333705740,333705740,415957,1077495790,1077495...",https://soundcloud.com/mackygee/make-me-feel-1,https://soundcloud.com/1077/sets/godtierplaylist


In [8]:
#Hàm lấy thông tin track truyền vào địa chỉ url và DataFrame tracks
def collect_track(url, tracks):
    session = HTMLSession()
    r = session.get(url)
    if(r):
        soup = BeautifulSoup(r.content,features="html.parser")
        value=[]
        for s in (soup.find_all("script")):
            #Xét script có dữ liệu cần tìm 
            if("hydratable" in str(s)):
                #Cắt chuỗi trong script
                i=str(s).find('{"hydratable":"sound"')
                s=str(s)[i+len('{"hydratable":"sound","data":{'):]
                j=s.find('"uri"')
                s=s[:j-1]
                #Cắt chuỗi thành một mảng gồm nhiều trường dữ liệu
                r1 = re.split(r',"',s)
                for i in r1:
                    #Với mỗi phần tử là trường dữ liệu tồn tại trong dataframe, thêm vào mảng tạm value theo thứ tự
                    t=re.split(r'":',i)
                    if(t[0].strip('"') in list(tracks)): 
                        value.append(t[1].strip('"'))
                #Lấy thêm thông tin người đăng track
                author =  soup.find("link",rel="author")
                if (author):
                    author = (author.get('href')).replace('/','')
                value.append(author)
                #Xét điều kiện kích thước của value bằng số lượng các cột của tracks thì thêm dữ liệu vào tracks
                if(len(value)==len(list(tracks))):
                    zipped = zip(list(tracks), value)
                    p = dict(zipped)
                    tracks=tracks.append(p,ignore_index=True)
    return tracks


In [9]:
#Kiểm tra hàm collect_track
url = f'https://soundcloud.com/jws-3-mj/last-summer?in=1041/sets/new-private-playlist'
t=pd.DataFrame(columns=t_unorder)
t = collect_track(url,t)
t=t[t_order]
display(t)

,id,title,description,track_format,duration,genre,label_name,tag_list,author,created_at,release_date,last_modified,comment_count,download_count,likes_count,playback_count,reposts_count,permalink_url
0,103051862,Last summer,,single-track,219840,Alternative,null,,jws-3-mj,2013-07-29T05:16:05Z,null,2014-11-26T15:58:53Z,30,0,2546,200081,183,https://soundcloud.com/jws-3-mj/last-summer


In [10]:
#Hàm thu thập cả 3 dữ liệu user, playlist, track và ghi vào file csv, trả về 3 dataframe
def collect_all(users_file, playlists_file, tracks_file,users,playlists,tracks):
    url = f'https://soundcloud.com/'
    index = 1002
    count = 0
    while (True):
        try:
            u_url=url+str(index)
            session = HTMLSession()
            u = session.get(u_url)
            if(u): 
                p_url = u_url+'/sets'
                session = HTMLSession()
                p=session.get(p_url) 
                #Lấy link playlists trong trang user hiện tại
                if(p.html.find('.audible', first=True)):
                    p_urls = p.html.find('.audible', first=True).absolute_links
                    for link in p_urls:
                        if('sets/' in link):
                            p_url=link
                            break
                    session = HTMLSession()
                    t = session.get(p_url)
                    t_soup=BeautifulSoup(t.content,features="html.parser")
                    try:
                        #Lấy link track đầu tiên trong playlist nếu có
                        t_url=f'https://soundcloud.com' + t_soup.find('h2',itemprop ='name').find('a',itemprop="url").get('href')
                        print(u_url)
                        tracks=collect_track(t_url,tracks)
                        playlists=collect_playlist(p_url,playlists)
                        users=collect_user(u_url,users)
                        #Tăng biến đếm lên 1
                        count=count+1 
                        #Xuất số lượng các bộ dữ liệu đã thu thập được ở hiện tại ra màn hình
                        print(count)
                    except:
                        index=index+1
                        continue 
            index=index+1
            #dieu kien dung
            if(count>1000): break
        except:
            time.sleep(1)
    #Sắp xếp lại các cột dữ liệu trong dataframe
    users=users[u_order]
    playlists=playlists[p_order]
    tracks=tracks[t_order]
    #Ghi dữ liệu ở dataframe vào file
    users.to_csv('users.csv',index=False)
    playlists.to_csv('playlists.csv',index=False)
    tracks.to_csv('tracks.csv',index=False)
    return users,playlists,tracks

In [11]:
#Khởi tạo DataFrame 
users=pd.DataFrame(columns=u_unorder)
playlists = pd.DataFrame(columns=p_unorder)
tracks=pd.DataFrame(columns=t_unorder)
#Thu thập dữ liệu và ghi vào file
users,playlists,tracks=collect_all('users.csv','playlists.csv','tracks.csv',users,playlists,tracks)
display()

https://soundcloud.com/1002
1
https://soundcloud.com/1014
2
https://soundcloud.com/1015
3
https://soundcloud.com/1018
4
https://soundcloud.com/1019
5
https://soundcloud.com/1023
6
https://soundcloud.com/1027
7
https://soundcloud.com/1033
8
https://soundcloud.com/1035
9
https://soundcloud.com/1037
10
https://soundcloud.com/1038
11
https://soundcloud.com/1041
12
https://soundcloud.com/1043
13
https://soundcloud.com/1045
14
https://soundcloud.com/1046
15
https://soundcloud.com/1052
16
https://soundcloud.com/1054
17
https://soundcloud.com/1060
18
https://soundcloud.com/1061
19
https://soundcloud.com/1065
20
https://soundcloud.com/1077
21
https://soundcloud.com/1079
22
https://soundcloud.com/1084
23
https://soundcloud.com/1086
24
https://soundcloud.com/1091
25
https://soundcloud.com/1096
26
https://soundcloud.com/1097
27
https://soundcloud.com/1101
28
https://soundcloud.com/1105
29
https://soundcloud.com/1106
30
https://soundcloud.com/1108
31
https://soundcloud.com/1118
32
https://soundclou

260
https://soundcloud.com/1833
261
https://soundcloud.com/1843
262
https://soundcloud.com/1846
263
https://soundcloud.com/1847
264
https://soundcloud.com/1848
265
https://soundcloud.com/1854
266
https://soundcloud.com/1863
267
https://soundcloud.com/1866
268
https://soundcloud.com/1867
269
https://soundcloud.com/1868
270
https://soundcloud.com/1873
271
https://soundcloud.com/1877
272
https://soundcloud.com/1884
273
https://soundcloud.com/1885
274
https://soundcloud.com/1887
275
https://soundcloud.com/1893
276
https://soundcloud.com/1896
277
https://soundcloud.com/1897
278
https://soundcloud.com/1901
279
https://soundcloud.com/1904
280
https://soundcloud.com/1909
281
https://soundcloud.com/1915
282
https://soundcloud.com/1922
283
https://soundcloud.com/1923
284
https://soundcloud.com/1928
285
https://soundcloud.com/1937
286
https://soundcloud.com/1943
287
https://soundcloud.com/1945
288
https://soundcloud.com/1947
289
https://soundcloud.com/1949
290
https://soundcloud.com/1952
291
http

https://soundcloud.com/2719
517
https://soundcloud.com/2720
518
https://soundcloud.com/2722
519
https://soundcloud.com/2724
520
https://soundcloud.com/2727
521
https://soundcloud.com/2730
522
https://soundcloud.com/2733
523
https://soundcloud.com/2736
524
https://soundcloud.com/2756
525
https://soundcloud.com/2759
526
https://soundcloud.com/2765
527
https://soundcloud.com/2770
528
https://soundcloud.com/2772
529
https://soundcloud.com/2784
530
https://soundcloud.com/2785
531
https://soundcloud.com/2788
532
https://soundcloud.com/2794
533
https://soundcloud.com/2797
534
https://soundcloud.com/2798
535
https://soundcloud.com/2800
536
https://soundcloud.com/2803
537
https://soundcloud.com/2804
538
https://soundcloud.com/2806
539
https://soundcloud.com/2810
540
https://soundcloud.com/2811
541
https://soundcloud.com/2812
542
https://soundcloud.com/2816
543
https://soundcloud.com/2822
544
https://soundcloud.com/2824
545
https://soundcloud.com/2826
546
https://soundcloud.com/2830
547
https://

773
https://soundcloud.com/3870
774
https://soundcloud.com/3879
775
https://soundcloud.com/3888
776
https://soundcloud.com/3898
777
https://soundcloud.com/3905
778
https://soundcloud.com/3919
779
https://soundcloud.com/3935
780
https://soundcloud.com/3938
781
https://soundcloud.com/3943
782
https://soundcloud.com/3944
783
https://soundcloud.com/3946
784
https://soundcloud.com/3947
785
https://soundcloud.com/3969
786
https://soundcloud.com/3971
787
https://soundcloud.com/3973
788
https://soundcloud.com/3975
789
https://soundcloud.com/3984
790
https://soundcloud.com/3990
791
https://soundcloud.com/3991
792
https://soundcloud.com/3999
793
https://soundcloud.com/4003
794
https://soundcloud.com/4007
795
https://soundcloud.com/4015
796
https://soundcloud.com/4019
797
https://soundcloud.com/4023
798
https://soundcloud.com/4031
799
https://soundcloud.com/4037
800
https://soundcloud.com/4046
801
https://soundcloud.com/4048
802
https://soundcloud.com/4049
803
https://soundcloud.com/4052
804
http

In [12]:
#Xuất dữ liệu được đọc từ file 
u=pd.read_csv('users.csv',index_col=[0])
p=pd.read_csv('playlists.csv',index_col=[0])
t=pd.read_csv('tracks.csv',index_col=[0])
display(u.head(),p.head(),t.head())

,username,first_name,last_name,full_name,description,avatar_url,country_name,city,created_at,last_modified,followers_count,followings_count,groups_count,comments_count,likes_count,playlist_likes_count,playlist_count,track_count,permalink_url
id,,,,,,,,,,,,,,,,,,,
81750389,G_Vaz,gustavo,Vaz,gustavo Vaz,NaN,https://i1.sndcdn.com/avatars-000244437617-37i...,undefined,NaN,2014-02-24T17:05:28Z,2017-01-12T23:09:15Z,18,60,0,19,110,11,7,0,https://soundcloud.com/1002
83249072,Mr. Popper,NaN,NaN,NaN,NaN,https://i1.sndcdn.com/avatars-000136608692-3ei...,undefined,NaN,2014-03-05T19:56:14Z,2017-05-15T17:28:14Z,4,6,0,1,140,4,2,0,https://soundcloud.com/1014
92963897,ahmed abdel aziz,ahmed,abdelaziz,ahmed abdelaziz,NaN,https://i1.sndcdn.com/avatars-000081941781-9mb...,Egypt,cairo,2014-05-04T21:57:17Z,2017-01-14T20:07:21Z,6,18,0,2,224,2,1,0,https://soundcloud.com/1015
138901086,LEESTER1018,NaN,NaN,NaN,NaN,https://i1.sndcdn.com/avatars-000208271769-qv2...,undefined,NaN,2015-02-16T18:33:04Z,2019-04-24T10:10:48Z,3,2,0,0,90,0,4,0,https://soundcloud.com/1018
7792201,LAKPUEDE77,ZAIDA,TA' PACHECO,ZAIDA TA' PACHECO,NaN,https://i1.sndcdn.com/avatars-000087461686-txt...,undefined,NaN,2011-10-01T23:37:15Z,2016-10-01T19:40:54Z,1,18,0,1,7,0,1,0,https://soundcloud.com/1019


,title,image_playlist,description,duration,genre,label_name,tag_list,author,created_at,release_date,last_modified,likes_count,reposts_count,public,sharing,numTracks,trackids,first_track,permalink_url
id,,,,,,,,,,,,,,,,,,,
259183402,Free Falling,https://i1.sndcdn.com/artworks-000177016371-ii...,NaN,2257469,NaN,NaN,NaN,G_Vaz,2016-09-14T16:29:35Z,NaN,2016-09-14T16:29:35Z,0,0,True,public,7,"'278534059,278534059,217168526,149145122,15708...",https://soundcloud.com/muzicarec/vinne-free-fa...,https://soundcloud.com/1002/sets/free-falling
81855177,corridos,https://i1.sndcdn.com/artworks-000004639182-of...,NaN,7086949,NaN,NaN,NaN,Mr. Popper,2015-02-17T15:24:09Z,NaN,2015-02-17T15:24:11Z,0,0,True,public,18,"'10143348,2776674,167937578,113699704,11876588...",https://soundcloud.com/freestyleent/trokero-lo...,https://soundcloud.com/1014/sets/corridos
58773046,zizo,https://i1.sndcdn.com/artworks-000042856160-sg...,NaN,4114186,NaN,NaN,NaN,ahmed abdel aziz,2014-11-10T14:00:13Z,NaN,2014-11-10T14:00:14Z,0,0,True,public,11,"'83090027,23757868,49390532,15403530,81914622,...",https://soundcloud.com/sama-ramadan/vbuxbjytnob3,https://soundcloud.com/1015/sets/zizo
82778512,ELECTRO,https://i1.sndcdn.com/artworks-000057083086-c1...,NaN,1261568,NaN,NaN,NaN,LEESTER1018,2015-02-20T18:54:05Z,NaN,2016-01-06T18:19:26Z,0,0,True,public,4,"'109032400,8539027,108382051,30000068'",https://soundcloud.com/edmdopestmusic/memories,https://soundcloud.com/1018/sets/electro
63241849,salsa,https://i1.sndcdn.com/artworks-000075308767-ij...,NaN,2351057,NaN,NaN,NaN,LAKPUEDE77,2014-12-09T23:35:34Z,NaN,2014-12-09T23:35:35Z,0,0,True,public,1,'142612946',https://soundcloud.com/dj-aneudy/salsa-dominic...,https://soundcloud.com/1019/sets/salsa


,title,description,track_format,duration,genre,label_name,tag_list,author,created_at,release_date,last_modified,comment_count,download_count,likes_count,playback_count,reposts_count,permalink_url
id,,,,,,,,,,,,,,,,,
278534059,VINNE - Free Falling (Original Mix),"Don’t be afraid to free falling, go ahead and ...",single-track,364990,Nu Disco,Muzica Records,NaN,muzicarec,2016-08-16T18:20:59Z,NaN,2018-02-11T01:15:40Z,10.0,0.0,1601.0,80678.0,178,https://soundcloud.com/muzicarec/vinne-free-fa...
10143348,Trokero Locochon - Gerardo Ortiz,NaN,single-track,139390,NaN,NaN,NaN,freestyleent,2011-02-06T04:16:32Z,NaN,2021-01-02T16:44:13Z,128.0,651.0,19105.0,3522703.0,1388,https://soundcloud.com/freestyleent/trokero-lo...
83090027,كل ده كان ليه - محمد محسن,NaN,single-track,339910,محمد محسن,NaN,NaN,sama-ramadan,2013-03-13T17:29:20Z,NaN,2021-05-17T18:21:12Z,571.0,100.0,92219.0,3269558.0,9394,https://soundcloud.com/sama-ramadan/vbuxbjytnob3
109032400,Strangers-Seven Lions with Myon and Shane 54 (...,I noticed that the last one I posted didnt upl...,single-track,273867,Trancestep\\ Trance\\ Dance\\ Groove,NaN,Strangers SevenLions Myon Shane54 Trancetep Tr...,edmdopestmusic,2013-09-05T23:18:44Z,NaN,2020-07-28T08:07:44Z,134.0,100.0,13976.0,1062778.0,3530,https://soundcloud.com/edmdopestmusic/memories
142612946,Salsa Dominicana Mix 40 Minutos - Abril 2014,Aqui les dejo una mezcla de salsa dominicana l...,single-track,2351057,Salsa,NaN,NaN,dj-aneudy,2014-04-02T00:38:11Z,NaN,2020-12-19T06:02:34Z,88.0,55733.0,12287.0,1418167.0,727,https://soundcloud.com/dj-aneudy/salsa-dominic...
